In [12]:
!pip install transformers


In [26]:
from google.colab import drive
drive.mount('/content/drive')

import json
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Load the saved model and tokenizer
model_path = "/content/drive/MyDrive/final_model.h5"  # Update with the actual path
checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

# Define the predefined annotations
predefined_annotations = [
    ["Feeling-bad-about-yourself-or-that-you-are-a-failure-or-have-let-yourself-or-your-family-down"],
    ["Feeling-down-depressed-or-hopeless"],
    ["Feeling-tired-or-having-little-energy"],
    ["Little-interest-or-pleasure-in-doing", "yes"],
    ["Moving-or-speaking-so-slowly-that-other-people-could-have-noticed-Or-the-opposite-being-so-fidgety-or-restless-that-you-have-been-moving-around-a-lot-more-than-usual"],
    ["Poor-appetite-or-overeating"],
    ["Thoughts-that-you-would-be-better-off-dead-or-of-hurting-yourself-in-some-way"],
    ["Trouble-concentrating-on-things-such-as-reading-the-newspaper-or-watching-television"],
    ["Trouble-falling-or-staying-asleep-or-sleeping-too-much"]
]

def predict_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.sigmoid(logits)

    threshold = 0.3
    predicted_labels = (predictions > threshold).cpu().numpy().astype(int).flatten()

    return predicted_labels

def process_input_json(input_json_path, output_text_path):
    with open(input_json_path, 'r') as input_file, open(output_text_path, 'w') as output_file:
        output_file.write("{\n\n")
        for entry in json.loads(input_file.read()):
            post_text = entry.get('post_text', '')
            post_title = entry.get('post_title', '')
            combined_text = f"{post_text} {post_title}"
            predicted_labels = predict_text(combined_text)
            annotations_with_labels = zip(predefined_annotations, ["no" if label == 0 else "yes" for label in predicted_labels])
            output_file.write(f"\t{list(annotations_with_labels)},\n")
        output_file.write("}\n\n\n")
        print("Result.txt is saved in the drive!")

if __name__ == "__main__":

    input_json_path = "/content/drive/MyDrive/test.json"
    output_text_path = "/content/drive/MyDrive/result.txt"
    process_input_json(input_json_path, output_text_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Result.txt is saved in the drive!
